In [1]:
from dataset_helpers import read_folder_filenames, is_dmso
from dataset_helpers import preprocess, sliding_window, get_label

from skimage.io import imread
import os
import numpy as np

%matplotlib inline
RS = 2211

### Loading filenames

In [2]:
## change this if going into a different hard drive
folder = '/Volumes/TARDIS/'

In [3]:
all_filenames = []
for ck in ["CK19", "CK21", "CK22"]:
    all_filenames.extend(read_folder_filenames(folder+ck))
all_filenames = sorted(all_filenames)

In [4]:
dmso_files = []
for file in all_filenames:
    if is_dmso(file):
        dmso_files.append(file)

In [5]:
CK19_files = sorted(read_folder_filenames(folder+"CK19"))
CK21_files = sorted(read_folder_filenames(folder+"CK21"))
CK22_files = sorted(read_folder_filenames(folder+"CK22"))

### Take into consideration before running:
* after passing a sliding window over 900 files of size 2048x2048 for a 192x192 window you will get about ~100,000 images
* there are much less DMSO images.
* but this is about the size of each of the CK19, CK21, CK22 folders.


In [6]:
def compress_combine_images(out, filenames, size):
    """
    returns:
    a npz file of:
     - image arrays in shape (size, size, 3)
        * channel R/0 = dendritic cell (red)
        * channel G/1 = t-cell (green)
        * channel B/2 = empty (0s)
     - filenames (unmodified)
    
        
    @parameters:
    out = name of the outputted compressed file
    filenames = all filenames of files to compress
    size = size of output images 
    
    
    @assumptions:
    * validity of filenames has been checked
    """
    
    compressed = []
    fn = []
    idx = 0 
    while idx < len(filenames):
        tcell = imread(filenames[idx])
        dcell = imread(filenames[idx+1])
        img = np.ndarray(shape=(tcell.shape[0], tcell.shape[1], 3))
        img[:,:,0] = dcell
        img[:,:,1] = tcell
        windows = sliding_window(img, size, rgb=True)
        for img in windows:
            compressed.append(img)
            fn.append(filenames[idx])
            
    compressed = np.array(compressed)
    fn = np.array(fn)
    np.savez(out, x=compressed, y=fn)
    
    print("All images combined and compressed into %s" % out)
    

In [ ]:
compress_combine_images("/Users/Leonore/Desktop/dmso_full.npz", dmso_files, 192)

In [1]:
def compress_images(out, filenames, size):
    """
    returns:
    a npz file of:
     - image arrays in shape (size, size, 1)
     - filenames (unmodified)
    
    @parameters:
    out = name of the outputted compressed file
    filenames = all filenames of files to compress
    size = size of output images 
    
    
    @assumptions:
    * validity of filenames has been checked
    """
    
    compressed = []
    fn = [] # right now for double checking would prefer to get label afterwards
    
    for file in filenames:
        img = imread(file)
        windows = sliding_window(img, size)
        for img in windows:
            new_img = img#preprocess(img)
            compressed.append(new_img)
            fn.append(file)
    
    compressed = np.array(compressed)
    fn = np.array(fn)
    np.savez(out, x=compressed, y=fn)
    
    print("All files compressed into %s" % out)

In [6]:
compress_images("/Volumes/TARDIS/dmso_unmodified.npz", dmso_files, 192)

All files compressed into /Volumes/TARDIS/dmso_unmodified.npz


In [31]:
np.savez("test.npz", x="xcontent", y="ycontent")

In [10]:
npzfile = np.load('/Users/Leonore/Desktop/dmso_unmodified.npz')

In [11]:
files = npzfile["y"]